---
## <font color="red">1 | SYSTÈME DE</font> RAISONNEMENT
---

Dans ce notebook, vous apprendrez à implémenter un système de raisonnement avec des connaissances de base sur le monde des meurtres.

### <font color="red">1.1 - Les</font> paquets

In [19]:
from aima.logic import *
import nltk
import speech_recognition as sr
from aima3.logic import *

### <font color="red">1.2 - Le moteur d'</font> inférence

In [20]:
# Permet d'inferer qui est le meurtrier, quand, comment, où il a tué.
class CrimeInference:

    def __init__(self):
        self.weapons = ["Corde", "Fusil", "Couteau", "Clé_anglaise", "Matraque", "Poison"]
        self.rooms = ["Phare", "Radio_room", "Embarcadère", "Plage", "Observatoire", "Serre", "Générateur"]
        self.persons = ["Governeur_White", "Botaniste_Green", "Docteur_Brown", "Capitaine_Gray", "Ingenieur_Blue", "Gardien_de_phare_Red", "Professeur_Yellow"]
        
        # Liste de clauses (faits) qui seront stockées dans la base de connaissance.
        self.clauses = []        
        
        self.base_clauses()
        self.initialize_KB()
        self.inference_rules()
        
        # Base de connaissances (First-order logic - FOL)
        self.crime_kb = FolKB(self.clauses)

    # Déclaration dans la logique du premier ordre
    def base_clauses(self):
        # Le paramètre est une arme
        self.arme_clause = 'Arme({})'
        
        # Le paramètre est une pièce
        self.piece_clause = 'Piece({})'
        
        # Le paramètre est une persone
        self.personne_clause = 'Personne({})'

        # paramètre 1 : arme; paramètre 2 : pièce
        # p.ex.: Le couteau se trouve dans la cuisine
        self.weapon_room_clause = 'Arme_Piece({},{})'

        # paramètre 1 : personne; paramètre 2 : pièce; paramètre 3 : heure
        # p.ex.: Mustart était dans la cuisine à 11h00
        self.person_room_hour_clause = 'Personne_Piece_Heure({}, {}, {})'

        # paramètre 1 : personne; paramètre 2 : piece
        # p.ex.: Mustard se trouve dans la cuisine
        self.person_room_clause = 'Personne_Piece({}, {})'

        # paramète 1 : personne
        # p. ex.: Mustard est mort
        self.dead_clause = 'EstMort({})'
        
        # paramète 1 : personne
        # p. ex.: Mustard est vivant
        self.alive_clause = 'EstVivant({})'

        # paramètre 1 : personne
        # p. ex.: Mustard est la victime
        self.victim_clause = 'Victime({})'

        # paramètre 1 : personne
        # p. ex.: Mustard a des marques au cou
        self.body_mark_clause = 'MarqueCou({})'

        # paramètre 1 : piece; paramètre 2 : piece
        self.room_different_clause = 'PieceDifferente({},{})'

        # paramètre 1 : piece; paramètre 2 : piece
        self.weapon_different_clause = 'ArmeDifferente({},{})'

        # paramètre 1 : heure
        self.crime_hour_clause = 'HeureCrime({})'

        # paramètre 1 : heure
        self.crime_hour_plus_one_clause = 'UneHeureApresCrime({})'

    def initialize_KB(self):
        # Clause pour differencier les pièces
        for i in range(len(self.rooms)):
            for j in range(len(self.rooms)):
                if i != j:
                    # Le bureau est different de la cuisine = PieceDifferente(Bureau, Cuisine)
                    self.clauses.append(expr(self.room_different_clause.format(self.rooms[i], self.rooms[j])))

        # Clause pour differencier les armes
        for i in range(len(self.weapons)):
            for j in range(len(self.weapons)):
                if i != j:
                    # Le couteau est different de la corde = ArmeDifferente(Couteau, Corde)
                    self.clauses.append(expr(self.weapon_different_clause.format(self.weapons[i], self.weapons[j])))

        # Initialiser KB sur Armes, Pieces, Personnes
        for weapon in self.weapons:
            # Le couteau est une arme = Arme(Couteau)
            self.clauses.append(expr(self.arme_clause.format(weapon)))

        for room in self.rooms:
            # La cuisine est une pièce = Piece(Cuisine)
            self.clauses.append(expr(self.piece_clause.format(room)))

        for person in self.persons:
            # Mustar est une personne = Personne(Mustard)
            self.clauses.append(expr(self.personne_clause.format(person)))
    
    # Expressions dans la logique du premier ordre permettant de déduire les caractéristiques du meurtre
    def inference_rules(self):
        # Determine la piece du crime
        self.clauses.append(expr('EstMort(x) & Personne_Piece(x, y) ==> PieceCrime(y)'))

        # Determiner l'arme du crime
        self.clauses.append(expr('PieceCrime(x) & Arme(y) & Piece_Arme(y, x) ==> ArmeCrime(y)'))
        self.clauses.append(expr("EstMort(x) & MarqueCou(x) ==> ArmeCrime(Corde)"))
        self.clauses.append(expr("EstMort(x) & MarqueFusil(x) ==> ArmeCrime(Fusil)"))
        self.clauses.append(expr("EstMort(x) & MarqueCouteau(x) ==> ArmeCrime(Couteau)"))
        self.clauses.append(expr("EstMort(x) & MarqueCleAnglaise(x) ==> ArmeCrime(Clé_anglaise)"))
        self.clauses.append(expr("EstMort(x) & MarqueMatraque(x) ==> ArmeCrime(Matraque)"))
        self.clauses.append(expr("EstMort(x) & MarquePoison(x) ==> ArmeCrime(Poison)"))

        # Si la personne est morte alors elle est la victime et ce n'est pas un suicide
        self.clauses.append(expr('EstMort(x) ==> Victime(x)'))

        # Si la personne est morte alors elle est innocente et ce n'est pas un suicide
        self.clauses.append(expr('EstMort(x) ==> Innocent(x)'))

        # Si la personne est vivante et était dans une pièce
        # qui ne contient pas l'arme du crime, alors elle est innocente
        self.clauses.append(expr(
            'EstVivant(p) & UneHeureApresCrime(h1) & Personne_Piece_Heure(p,r2,h1) & PieceCrime(r1)'
            ' & PieceDifferente(r1,r2) & ArmeCrime(a1) & Arme_Piece(a2,r2) & ArmeDifferente(a1,a2) ==> Innocent(p)'))

        # Si la personne se trouvait dans une piece qui contient l'arme
        # qui a tué la victime une heure après le meurtre alors elle est suspecte
        self.clauses.append(expr(
            'EstVivant(p) & UneHeureApresCrime(h1) & Personne_Piece_Heure(p,r2,h1) & PieceCrime(r1)'
            ' & PieceDifferente(r1,r2) & ArmeCrime(a) & Arme_Piece(a,r2) ==> Suspect(p)'))

    # Ajouter des clauses, c'est-à-dire des faits, à la base de connaissances
    def add_clause(self, clause_string):
        self.crime_kb.tell(expr(clause_string))

    # Demander à la base de connaissances qui est la victime
    def get_victim(self):
        result = self.crime_kb.ask(expr('Victime(x)'))
        if not result:
            return False
        else:
            return result[x]
        
    # Demander à la base de connaissances la pièce du meurtre
    def get_crime_room(self):
        result = self.crime_kb.ask(expr('PieceCrime(x)'))
        if not result:
            return False
        else:
            return result[x]

    # Demander à la base de connaissances l'arme du meurtrier
    def get_crime_weapon(self):
        result = self.crime_kb.ask(expr('ArmeCrime(x)'))
        if not result:
            return result
        else:
            return result[x]

    # Demander à la base de connaissances l'heure du meurtre
    def get_crime_hour(self):
        result = self.crime_kb.ask(expr('HeureCrime(x)'))
        if not result:
            return result
        else:
            return result[x]

    def get_crime_hour_plus_one(self):
        result = self.crime_kb.ask(expr('UneHeureApresCrime(x)'))
        if not result:
            return result
        else:
            return result[x]
    
    # Demander à la base de connaissances le suspect
    def get_suspect(self):
        result = self.crime_kb.ask(expr('Suspect(x)'))
        if not result:
            return result
        else:
            return result[x]

    # Demander à la base de connaissances la liste d'innocents
    def get_innocent(self):
        result = list(fol_bc_ask(self.crime_kb, expr('Innocent(x)')))
        res = []

        for elt in result:
            if not res.__contains__(elt[x]):
                res.append(elt[x])
        return res

### <font color="red">1.3 - Les faits et </font> la déduction
---
Maintenant que nous avons le moteur de déduction, il est temps de commencer à alimenter la base de connaissances avec les faits du monde.

Comme nous communiquerons les faits au moteur d’inférence en français, plusieurs grammaires ont été créées pour interpréter les phrases. Celles-ci se trouvent dans le dossier `grammars`.

À partir des grammaires, nous pouvons analyser les phrases et puis les transformer en expressions logiques du premier ordre. C'est à cela que servent les fonctions `results_as_string` et `to_fol`. 

In [21]:
# Cette fonction retourne le format d'une expression logique de premier ordre
def results_as_string(results):
    res = ''
    for result in results:
        # synrep = syntactic representation
        # semrep = semantic representation
        for (synrep, semrep) in result:            
            res += str(semrep)
    return res

La fonction `nltk.interpret_sents` donne une représentation de premier ordre d'un fait en fraçais à partir d'une grammaire. La représentation est récupérée et transmise à la fonction `results_as_string` pour la formater sous la forme Exp(x). Enfin, l'expression mise en forme est renvoyée pour l'ajouter à la base de connaissances.

In [22]:
# Cette fonction transforme une phrase en fraçais dans une expression logique du premier ordre
def to_fol(fact, grammar):
    sent = results_as_string(nltk.interpret_sents(fact, grammar))
    print(sent)
    return sent    

Nous créons maintenant une instance du moteur d'inférence et commençons à ajouter les faits à la base de connaissances. Les faits sont communiqués en français, interprétés par la grammaire appropriée et ajoutés.

In [23]:
agent = CrimeInference()

In [24]:
# Faits
facts = [['Scarlet est morte'],
        ['Mustard est vivant'],
        ['Peacock est vivant'],
        ['Plum est vivant'],
        ['White est vivant']]

# Les fait sont ajoutés à la base de connaissances
agent.add_clause(to_fol(facts[0], 'grammars/personne_morte.fcfg'))
facts.pop(0)

for fact in facts:    
    agent.add_clause(to_fol(fact, 'grammars/personne_vivant.fcfg'))

ValueError: Grammar does not cover some of the input words: "'Scarlet'".

In [ ]:
# On se rend compte que Scarlet est morte par étranglement
fact = ['Scarlet a des marques au cou']
agent.add_clause(to_fol(fact, 'grammars/personne_marque.fcfg'))

fact = ['Scarlet est dans le bureau']
agent.add_clause(to_fol(fact, 'grammars/personne_piece.fcfg'))

# On sait que Peacock est dans le bureau
fact = ['Peacock est dans le bureau']
agent.add_clause(to_fol(fact, 'grammars/personne_piece.fcfg'))

# Demande à Peacock l'heure du decès -> Rep : 14h
fact = ['Scarlet est morte à 14h']
agent.add_clause(to_fol(fact, 'grammars/personne_morte_heure.fcfg'))

uneHeureApres = agent.get_crime_hour() + 1

agent.add_clause('UneHeureApresCrime({})'.format(uneHeureApres))

# Demande à Peacock dans quelle pièce il était une heure après le meurtre -> Rep : Peacock dans le Salon à 15h
fact = ['Peacock était dans le salon à ' + str(uneHeureApres) + 'h']
agent.add_clause(to_fol(fact, 'grammars/personne_piece_heure.fcfg'))

In [ ]:
# Dans le salon

# Voit qu'il y a un fusil et Plum dans le salon
fact = ['Le fusil est dans le salon']
agent.add_clause(to_fol(fact, 'grammars/arme_piece.fcfg'))

fact = ['Plum est dans le salon']
agent.add_clause(to_fol(fact, 'grammars/personne_piece.fcfg'))

# Demande à Plum dans quelle pièce il était une heure après le meurtre -> Rep : Plum dans le Salon à 15h
fact = ['Plum était dans le salon à ' + str(uneHeureApres) + 'h']
agent.add_clause(to_fol(fact, 'grammars/personne_piece_heure.fcfg'))

In [ ]:
# Dans la cuisine

# Voit qu'il y a un couteau, White et Mustard dans la cuisine
fact = ['Le couteau est dans la cuisine']
agent.add_clause(to_fol(fact, 'grammars/arme_piece.fcfg'))

fact = ['White est dans la cuisine']
agent.add_clause(to_fol(fact, 'grammars/personne_piece.fcfg'))

fact = ['Mustard est dans la cuisine']
agent.add_clause(to_fol(fact, 'grammars/personne_piece.fcfg'))

# Demande à White dans quelle pièce il était une heure après le meurtre -> Rep : White dans la Cuisine à 15h
fact = ['White était dans la cuisine à ' + str(uneHeureApres) + 'h']
agent.add_clause(to_fol(fact, 'grammars/personne_piece_heure.fcfg'))

# Demande à Mustard dans quelle pièce il était une heure après le meurtre -> Rep : Mustard dans le Garage à 15h
fact = ['Mustard était dans le garage à ' + str(uneHeureApres) + 'h']
agent.add_clause(to_fol(fact, 'grammars/personne_piece_heure.fcfg'))

In [ ]:
# Dans le garage

# On se rend compte qu'il y a une corde dans le garage
fact = ['La corde est dans le garage']
agent.add_clause(to_fol(fact, 'grammars/arme_piece.fcfg'))

Enfin, après avoir fourni tous les faits (scénario), nous demandons au moteur d’inférence de tirer ses conclusions.

In [ ]:
# Conclusions
print("Pièce du crime : ", agent.get_crime_room())
print("Arme du crime : ", agent.get_crime_weapon())
print("Personne victime : ", agent.get_victim())
print("Heure du crime : ", agent.get_crime_hour())
print("Meurtrier : ", agent.get_suspect())
print("Personnes innocentes : ", agent.get_innocent())

# Reconnaissance vocale


In [31]:
import speech_recognition as sr

def reconnaitre_parole():
    r = sr.Recognizer()
    with sr.Microphone() as source:
        print("Parlez maintenant...")
        r.adjust_for_ambient_noise(source)  # réduit le bruit ambiant
        audio = r.listen(source)

    try:
        texte = r.recognize_google(audio, language="fr-FR")
        print("Vous avez dit : " + texte)
    except sr.UnknownValueError:
        print("Je n'ai pas compris.")
    except sr.RequestError as e:
        print("Erreur avec le service Google : {0}".format(e))

# Test
# reconnaitre_parole()


🎤 Parlez maintenant...
✅ Vous avez dit : ceci est un test mon nom est Annie
